In [1]:
import numpy as np
import miniball
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

from ML2_lib import loss
from ML2_lib import noise
from ML2_lib import algo_sgd
from  ML2_lib import merge
from ML2_lib import DC_SGD

In [2]:
# データ生成デモ Gauss
X_mean = 5
X_var = 2
w_star = np.array([1])
w_star = w_star.reshape(1,-1)
d = w_star.shape[1]
n = 1000
rng = np.random.default_rng()
X = rng.normal(loc=X_mean,size=(n,d),scale=X_var)
E = rng.normal(loc=0,scale=1,size=(1,n))
Y = np.dot(w_star,X.T) + E



In [3]:

a = loss.LinearQuadraticLoss()
w_init = np.zeros(d)
data = [X,Y.T]
lr = 0.01
core = algo_sgd.SGD(w_init=w_init,a=lr,t_max=n-1,data=data)
for _ in core:
    core.update(a)


In [4]:

core.w


array([1.03367851])

In [5]:

trial_num = 10 #試行回数
result_gauss = [] #パラメータの最終結果　トライアル数*分割数k*特徴量次元
result_loss_gauss = [] # 過剰期待損失の最終結果　トライアル数*分割数k
valid_k = 100 # 分割数　1 ~ valid_k

X_mean = 5
X_var = 2
a = 0.01 # 学習率
c = 5 # wの初期値がとりうる範囲
n = 250 # データセットのサンプル数
b = 2.2 # ノイズの分散
w_star = np.array([1,2,3,4,5,6])
w_star = w_star.reshape(1,-1)
d = w_star.shape[1]

for _ in tqdm(range(trial_num)):
    w_init = w_star + rng.uniform(-c,c,size=d)

    w_trial = [] # モデルを貯めていく、必要かどうか
    loss_store = [] # 過剰期待損失を貯めていく
    for k in range(1,valid_k+1):
        core_store = []
        model_store = []
        son = loss.LinearQuadraticLoss()
        for i in range(k):
            core_num = n // k
            rng = np.random.default_rng()
            X = rng.normal(loc=X_mean,size=(n,d),scale=X_var)
            E = rng.normal(loc=0,scale=b,size=(1,n))
            Y = np.dot(w_star,X.T) + E
            data = [X,Y.T]

            core = algo_sgd.SGD(w_init=w_init,a=a,t_max=core_num-1,data=data)
            for _ in core:
                core.update(son)
            core_store.append(core)
            model_store.append(core.w)


        w_dc,_ = miniball.get_bounding_ball(np.array(model_store).reshape((-1,d)), epsilon=1e-7)
        w_dc = w_dc.reshape(1,-1)
        w_trial.append(w_dc)
        # 過剰期待損失　E[(<(w-w^*),X>)^2]　Xが正規分布の場合　E[X^2] = X_mean^2 + X_var^2 * 単位行列
        E_X = np.diag(np.ones(w_dc.shape[0])* (X_var**2)  + (X_mean**2))
        excess_risk = np.dot(np.dot(E_X,w_dc),w_dc.T) + np.dot(np.dot(E_X,w_star),w_star.T) - 2 * np.dot(np.dot(E_X,w_star),w_dc.T)
        loss_store.append(excess_risk[0][0])
    result_gauss.append(np.array(w_trial))
    result_loss_gauss.append(np.array(loss_store))

result_gauss = np.array(result_gauss)
result_loss_gauss = np.array(result_loss_gauss)


  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
fdic = {
    "size" : 20,
}

columns=['1', '2', '4','5','10','20','40','50']
fig = plt.figure(figsize=(10.0, 8.0))
ax1 = fig.add_subplot(111)

bp1 = ax1.boxplot(result_loss_gauss[:,[0,1,3,4,9,19,39,49]])
ax1.set_xticklabels(columns,fontsize=20)
ax1.set_title('R(w)-R^*',fontsize=20)
ax1.set_xlabel('k',fontdict=fdic)

plt.show()

In [ ]:
df = pd.DataFrame(result_loss_gauss[:,[0,1,3,4,9,19,39,49]],columns=columns)
df.describe()

In [ ]:
model_store

In [ ]:
rng = np.random.default_rng()


In [ ]:
rng.normal()

In [ ]:
son = loss.LinearQuadraticLoss()
fuga = DC_SGD.DCSGD(w_star=w_star,n=n,E_var=2.2,X_mean=X_mean,X_var=X_var,noise="normal",loss_type=son,c=c)